In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from mnist import MNIST
from emnist import extract_training_samples, list_datasets, extract_test_samples

In [2]:
train_images, train_labels = extract_training_samples('byclass')

In [3]:
test_images, test_labels = extract_test_samples('byclass')

In [4]:
train_images = tf.keras.utils.normalize(train_images, axis = 1)
test_images = tf.keras.utils.normalize(test_images, axis = 1)

train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

In [5]:
train_datagen = ImageDataGenerator(rotation_range=15,
                             width_shift_range=0.10,
                             height_shift_range=0.10)

train_datagen.fit(train_images.reshape(train_images.shape[0], 28, 28, 1))

In [6]:
val_datagen = ImageDataGenerator()
val_datagen.fit(test_images.reshape(test_images.shape[0], 28, 28, 1))

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(62, activation = 'softmax')
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 128)        0

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [10]:
history = model.fit(train_datagen.flow(train_images, train_labels, batch_size=1024),
         validation_data= val_datagen.flow(test_images, test_labels,
         batch_size=32), epochs= 20)

Epoch 1/20
682/682 [==============================] - 175s 243ms/step - loss: 1.0864 - accuracy: 0.6832 - val_loss: 0.4804 - val_accuracy: 0.8325
Epoch 2/20
682/682 [==============================] - 166s 243ms/step - loss: 0.5614 - accuracy: 0.8079 - val_loss: 0.4354 - val_accuracy: 0.8445
Epoch 3/20
682/682 [==============================] - 161s 237ms/step - loss: 0.4990 - accuracy: 0.8252 - val_loss: 0.4074 - val_accuracy: 0.8522
Epoch 4/20
682/682 [==============================] - 160s 235ms/step - loss: 0.4682 - accuracy: 0.8340 - val_loss: 0.3898 - val_accuracy: 0.8572
Epoch 5/20
682/682 [==============================] - 160s 235ms/step - loss: 0.4466 - accuracy: 0.8402 - val_loss: 0.3827 - val_accuracy: 0.8592
Epoch 6/20
682/682 [==============================] - 163s 239ms/step - loss: 0.4324 - accuracy: 0.8440 - val_loss: 0.3728 - val_accuracy: 0.8620
Epoch 7/20
682/682 [==============================] - 169s 247ms/step - loss: 0.4212 - accuracy: 0.8469 - val_loss: 0.3778 -

In [12]:
model.save('car_number_model.h5')